Install Requirements

In [1]:
%tensorflow_version 1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [ ]:
%cd
!git clone --quiet https://github.com/matterport/Mask_RCNN.git
%cd ~/Mask_RCNN
!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

Clone Repository

In [ ]:
!ssh-keygen -t rsa -b 4096  # generate ssh key for cloning

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
!cat /root/.ssh/id_rsa.pub

In [ ]:
%cd
!ssh -T git@github.com
# !git clone --quiet git@github.com:joangog/object-detection.git
!git clone --quiet https://github.com/joangog/object-detection.git
%cd ~/object-detection/mask_rcnn
!gdown --id 1XSoq8VSnuEcdeVbuWec2m8iVFWtw_6lC

Run Mask R-CNN

In [3]:
import os, sys

import numpy as np
from IPython.display import display, Javascript
from google.colab.output import eval_js

%cd ~/Mask_RCNN
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

%cd ~/object-detection
from mask_rcnn.visualize import display_instances

%cd ~/object-detection/mask_rcnn

# Directory of the project
root_dir = os.path.abspath("../")

# Directory of test images
image_dir = os.path.join(root_dir, "images")

# Directory of model
model_dir = os.path.join(root_dir, "mask_rcnn")

# Define model configurations
class MyConfig(Config):
    NAME = "coco"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 80  # COCO has 80 classes

# Define model
model = MaskRCNN(mode='inference', model_dir='./log', config=MyConfig())

# Load pre-trained COCO weights
model.load_weights(os.path.join(model_dir, "mask_rcnn_coco.h5"), by_name=True)

# Define COCO class names
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


def imageData2Array(imageData):
  # Receives JavaScript's ImageData and converts to Python Array
  
  data = imageData[0]  # It is in dictionary format
  height = imageData[1]
  width = imageData[2]

  # Initialize array
  array = np.zeros((height,width,3))  # 3 is for RGB (exclude Aplha)
  
  for i in range(0,height):  # For every line
    for j in range(0,width):  # For every column
      for k in range(0,3):  # For RGB
        array[i,j,k] = int(data[str(k + j*4 + i*width)])  # 4 is because RGB and Alpha is stored
  
  return array


def array2ImageData(array):
  # Receives Python Array and converts to JavaScript's ImageData
  shape =  array.shape
  height = shape[0]
  width = shape[1]

  # Initialize data
  data = [0 for i in range(4*height*width)]
  
  for i in range(0,height):  # For every line
    for j in range(0,width):  # For every column
      for k in range(0,3):  # For RGB
        data[k + j*4 + i*width] = int(array[i,j,k])

  return [data,height,width]


try:
  js = Javascript('''

    async function streamVideo() {
  
      const div = document.createElement('div');
      
      // Video components
      const video_input = document.createElement('video');
      video_input.id = 'video-input'
      video_input.style.display = 'block';
      video_input.style.float = 'left'
      const video_output = document.createElement('video');
      video_output.id = 'video-output'
      video_output.style.display = 'block';
      video_output.style.float = 'right'
      
      const stream_input = await navigator.mediaDevices.getUserMedia({video: true});
      video_input.srcObject = stream_input;

      const height = stream_input.getTracks()[0].getSettings().height
      const width = stream_input.getTracks()[0].getSettings().width

      document.body.appendChild(div);
      div.appendChild(video_input);
      div.appendChild(video_output);

      // Canvas components
      const canvas_input = document.createElement('canvas');
      canvas_input.id = 'canvas-input'
      canvas_input.style.display = 'block';
      canvas_input.width = width
      canvas_input.height = height
      const canvas_output = document.createElement('canvas');
      canvas_output.id = 'canvas-output'
      canvas_output.style.display = 'block';
      canvas_output.width = width
      canvas_output.height = height
      
      const stream_output = canvas_output.captureStream();
      video_output.srcObject = stream_output;

      div.appendChild(canvas_input);
      div.appendChild(canvas_output);
      
      await video_input.play();

      // Resize the output to fit the video element
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

    }

    async function readFrame() {
      video_input = document.getElementById('video-input')
      canvas_input = document.getElementById('canvas-input')
      canvas_input.getContext('2d').drawImage(video_input, 0, 0)
      imageData = canvas_input.getContext('2d').getImageData(0,0,canvas_input.height,canvas_input.width)
      data = imageData.data
      height = imageData.height
      width = imageData.width
      return [data,height,width]
    }

    async function writeFrame(imageArray) {
      data = new Uint8ClampedArray(imageArray[0])
      height = imageArray[1]
      width = imageArray[2]
      imageData = new ImageData(data,height,width)
      canvas_output = document.getElementById('canvas-output')
      canvas_output.getContext('2d').putImageData(imageData,0,0); 
    }

    ''')

except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

display(js)
eval_js('streamVideo({})')

# Run detection
while True: # loop over frames from the web camera
    
    data_in = eval_js('readFrame({})')
    frame = imageData2Array(data_in)
    
    # Run algorithm
    results = model.detect([frame], verbose=1)

    # Render masked frame
    masked_frame = display_instances(frame, results[0]['rois'], results[0]['masks'], results[0]['class_ids'],
                                class_names, results[0]['scores'])
    
    data_out = array2ImageData(masked_frame)
    frame = eval_js('writeFrame({})'.format(data_out))

ModuleNotFoundError: No module named 'google.colab'